1\. **Radioactive decay chain**

${\rm Tl}^{208}$ decays to ${\rm Pb}^{208}$ with a half-lieve of 3.052 minutes. Suppose to start with a sample of 1000 Thallium atoms and 0 of Lead atoms.

* Take steps in time of 1 second and at each time-step decide whether each Tl atom has decayed or not, accordingly to the probability $p(t)=1-2^{-t/\tau}$. Subtract the total number of Tl atoms that decayed at each step from the Tl sample and add them to the Lead one. Plot the evolution of the two sets as a function of time  
* Repeat the exercise by means of the inverse transform method: draw 1000 random numbers from the non-uniform probability distribution $p(t)=2^{-t/\tau}\frac{\ln 2}{\tau}$ to represent the times of decay of the 1000 Tl atoms. Make a plot showing the number of atoms that have not decayed as a function of time

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as rnd

In [ ]:
samples = 1000
Tl_atoms = 1000
Pb_atoms = 0
time_int = 2000

tau = 3.052*60
prob = 1 - 2**(-1/tau)

Tl_evolution = []
Pb_evolution = []

for t in range (time_int):
    Tl_evolution.append(Tl_atoms)
    Pb_evolution.append(Pb_atoms)
    Tl = np.random.uniform(0,1,size=Tl_atoms)
    Tl_atoms = len(Tl[Tl > prob])
    Pb_atoms += len(Tl[Tl <= prob])
        
plt.plot(np.linspace(0,time_int,time_int), Tl_evolution, label='Thallium Evolution')
plt.plot(np.linspace(0,time_int,time_int), Pb_evolution, label='Lead Evolution')
plt.legend()
plt.xlabel('Time [s]')
plt.ylabel('Number of atoms')
plt.show()

# MISS 2 POINT

2\. **Rutherford Scattering**

The scattering angle $\theta$ of $\alpha$ particles hitting a positively charged nucleus of a Gold atom ($Z=79$) follows the rule:

$$
\tan{\frac{1}{2} \theta} = \frac{Z e^2} {2\pi \epsilon_0 E b}
$$

where $E=7.7$ MeV and $b$ beam is the impact parameter. The beam is represented by a 2D gaussian distribution with $\sigma=a_0/100$ for both coordinates ($a_0$ being the Bohr radius). Assume 1 million $\alpha$ particles are shot on the gold atom.

Computing the fraction of particles that "bounce back",i.e. those particle whose scattering angle is greater than $\pi/2$ (which set a condition on the impact parameter $b$)

In [ ]:
n = 10**6
Z = 79
e = 1.60217*(10**-19)
E = 7.7*(10**6)*e
epsilon_0 = 8.85419*10**(-12)
bohr_radius = 5.29*10**(-11)

b = np.random.normal(scale=bohr_radius/100, size=n)
theta = 2*np.arctan(Z*e**2 / (2*np.pi*epsilon_0*E*b))

bounce_back = theta[theta > np.pi/2]
print("TFraction of particles that bounces back:", len(bounce_back)/n)

3\. **Monte Carlo integration: hit/miss vs mean value method**

Consider the function 

$$f(x) =\sin^2{\frac{1}{x(2-x)}}$$

* Compute the integral of $f(x)$ between 0 and 2 with the hit/miss method. Evaluate the error of your estimate
* Repeat the integral with the mean value method. Evaluate the error and compare it with the previous one

In [ ]:
from scipy import integrate

def f(x):
    return (np.sin(1/(x*(2-x))))**2

N = 10**5
hit=0
x = np.linspace(0,2,N)

real_value = integrate.quad(f, 0, 2, limit=10000)[0]
print("Real value: ", real_value)

for i in range(N):
    x = 2*np.random.random()
     # y < f(x)
    if np.random.random() < f(x):
        hit += 1
        
value_hm = 2*hit/N
print('Value with hit/miss: ', value_hm )
print('Error: ', abs(real_value-value_hm))

mean_value = 2 * f(2*np.random.random(N)).sum() / N
print('Mean value: ', mean_value )
print('Error: ', abs(real_value-mean_value))

4\. **Monte Carlo integration in high dimension**

* Start of by computing the area of a circle of unit radius, by integrating the function 

$$
f(x,y)=
\left\{
\begin{array}{ll}
      1 & x^2+y^2\le 1 \\
      0 & {\rm elsewhere}
\end{array} 
\right.
$$

* Generalize the result for a 10D sphere



In [ ]:
def area(x, y):
    if x**2+y**2<1:
        return 1
    else:
        return 0
    
N = 10**5
points = 2*np.random.random(size=(N, 2))-1
c = 0

for x,y in points:
    c+=area(x,y)

area = 4 * c / N
print("Estimated area of a circle: ", area)

# General case
def sphere_volume(p):
    squares = p**2
    if squares.sum()<1:
        return 1
    else:
        return 0
    
points = 2*np.random.random(size=(N, 10))-1
c = 0

for p in points:
    c+=sphere_volume(p)
    
area_md = 2**10*c/N
print("Estimated area of a 10D sphere: ", area_md)

5\. **Monte Carlo integration with importance sampling** 

Calculate the value of the integral:

$$
I=\int_0^1 \frac{x^{-1/2}}{e^x+1} dx
$$

using the importance sampling method with $w(x)=1/\sqrt{x}$. You should get a result about 0.84

In [ ]:
def f(x):
    return (x**(-1/2))/(np.exp(x)+1)

def w(x):
    return 1/(2*np.sqrt(x))

N = 10**5

x = np.random.random(size=N)
I = 1.0 / N * np.sum(f(x) / w(x))
print("Estimated Integral: ",I)